In [1]:
import requests
import os
import logging
import gdown
import random

import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms, models
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchinfo import summary
from torchvision.datasets import VOCSegmentation
import torchmetrics
import torchvision
import albumentations as A

import re
import string
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
import cv2
from PIL import Image
from tqdm import tqdm
import torchvision.transforms as T
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from scipy.io import loadmat
from sklearn.manifold import TSNE
from torchmetrics.classification import MulticlassF1Score, JaccardIndex, MulticlassPrecision, MulticlassRecall, MulticlassAveragePrecision
import pandas as pd
from torchinfo import torchinfo

from transformers import ConvNextV2Model, BertModel, BertTokenizer, ViTModel, ViTConfig
from transformers import AutoTokenizer, AutoModel, RobertaTokenizer, CLIPModel, CLIPTokenizer, CLIPProcessor
from transformers import DeiTConfig, DeiTFeatureExtractor, DeiTImageProcessor, DeiTModel
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import timm
from transformers import AlignModel, AlignProcessor, AlignConfig, AlignVisionConfig, AlignTextConfig
from transformers import CLIPModel, CLIPProcessor, CLIPConfig, CLIPTextConfig, CLIPVisionConfig, CLIPImageProcessor

from torchmetrics.functional import pairwise_cosine_similarity
from torchmetrics.classification import MultilabelAccuracy

In [2]:
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
logging.basicConfig(level=logging.ERROR)

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda:0")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
    
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")




There are 1 GPU(s) available.
We will use the GPU: NVIDIA GeForce RTX 3090


In [3]:
ROOT_DIR = '../Datasets/ocular-disease-recognition-odir5k/'

## DeiT

In [4]:
BATCH_SIZE = 32

In [5]:
CSV_PATH = ROOT_DIR + 'dataset_single_eye.csv'
TEST_CSV = ROOT_DIR + 'TESTING_dataset_single_eye.csv'
IMG_PATH = ROOT_DIR + 'preprocessed_images/'

In [6]:
torchvision.io.read_image(IMG_PATH + '0_left.jpg').shape

torch.Size([3, 512, 512])

In [7]:
train_val_df = pd.read_csv(CSV_PATH)
test_df = pd.read_csv(TEST_CSV)

In [8]:
def preprocess_text(df:pd.DataFrame):
    df['Keywords'] = df['Keywords'].str.lower()
    df['Keywords'] = df['Keywords'].apply(lambda x: " ".join(x.split()))
    df['Keywords'] = df['Keywords'].apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation), '' , x))
    return df
train_val_df = preprocess_text(train_val_df)
test_df = preprocess_text(test_df)

In [9]:
np.max(train_val_df['Keywords'].apply(lambda x: len(x.split())))

10

In [10]:
train_df, val_df = train_test_split(train_val_df, test_size = 0.15, random_state= 123456)
len(train_df), len(val_df)

(4877, 861)

In [11]:
IMG_SIZE = (224, 224)

rescale_transform = torchvision.transforms.Compose([
    #torchvision.transforms.CenterCrop(IMG_SIZE),
    torchvision.transforms.Resize(IMG_SIZE, antialias = False, interpolation = torchvision.transforms.InterpolationMode.NEAREST),
    torchvision.transforms.Normalize(
        timm.data.constants.IMAGENET_DEFAULT_MEAN,
        timm.data.constants.IMAGENET_DEFAULT_STD
    )
])

augmentation = torchvision.transforms.Compose([
    torchvision.transforms.RandomHorizontalFlip(p = 0.5),
    torchvision.transforms.RandomVerticalFlip(p= 0.5),
    #torchvision.transforms.RandomRotation(90)
])

In [12]:
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
#torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'bert-base-uncased')
extractor = CLIPImageProcessor.from_pretrained('openai/clip-vit-base-patch32')

In [13]:
class ODIRDataset(Dataset) :
    def __init__(self, df, IMG_FOLDER, tokenizer = processor, feature_extractor = extractor, augmentation = None) :
        '''
        id : list of samples ids as string
        '''
        #self.text = [tokenizer(text = x, padding = 'max_length', max_length = 40, truncation = True, return_tensors = 'pt') for x in df['Keywords']]
        self.eye = df['eye']
        self.labels = torch.tensor(df[['N', 'D', 'G', 'C', 'A', 'H', 'M', 'O']].to_numpy()).float()
        self.img_dir = [IMG_PATH + x for x in df['Image']]

        self.augmentation = augmentation

        self.images = [extractor(Image.open(x), return_tensors='pt')['pixel_values'][0] for x in self.img_dir]
        #self.transform = transform
        
    def __len__(self):
        return len(self.images)
        
    def __getitem__(self, idx):
        batch_imgs = self.images[idx]
        if(self.augmentation is not None):
            batch_imgs = self.augmentation(batch_imgs)
        return batch_imgs, self.text[idx], self.labels[idx]

In [14]:
# train_dataset = ODIRDatasetMM(train_df, IMG_PATH)#, augmentation = augmentation)
# val_dataset   = ODIRDatasetMM(val_df, IMG_PATH)
# test_dataset  = ODIRDatasetMM(test_df, IMG_PATH)

In [15]:
# with open("datasets.trch", 'wb') as f:
#     torch.save([train_dataset, val_dataset, test_dataset], f)

In [16]:
with open("datasets.trch", 'rb') as f:
    train_dataset, val_dataset, test_dataset = torch.load(f)

In [17]:
train_dataloader = DataLoader(train_dataset, batch_size= BATCH_SIZE, shuffle = True)
val_dataloader = DataLoader(val_dataset, batch_size = BATCH_SIZE)
test_dataloader = DataLoader(test_dataset, batch_size= BATCH_SIZE)

In [18]:
# contrastive learning on training data finetuning

In [19]:

#odict_keys(['loss', 'logits_per_image', 'logits_per_text', 'text_embeds', 'image_embeds', 'text_model_output', 'vision_model_output'])

In [20]:
class ClassificationNet(nn.Module):
    def __init__(self, dims = 640, drop_prob = 0.5):
        super().__init__()
        self.layer_1 = nn.Linear(dims, 768)
        self.layer_2 = nn.Linear(768, 768)
        self.layer_3 = nn.Linear(768, dims)
        
        self.dropout = nn.Dropout(drop_prob)
        
    def forward(self, input):
        resi = input
        out = self.dropout(input)
        out = F.relu(self.layer_1(out))
        out = self.dropout(out)
        out = F.relu(self.layer_2(out))
        out = self.dropout(out)
        out = F.relu(self.layer_3(out))
        out = out + resi
        return out


In [21]:
class ContrastiveLearning(nn.Module):
    def __init__(self):
        super().__init__()
        self.align = CLIPModel(CLIPConfig())
        self.text_net = ClassificationNet(dims = 512, drop_prob=0.5)
        self.text_head = nn.Linear(512, 8)
        self.img_net = ClassificationNet(dims = 512, drop_prob=0.5)
        self.img_head = nn.Linear(512, 8)
    
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, pixel_values, input_ids, attention_mask):
        out = self.align(pixel_values = pixel_values, input_ids = input_ids, attention_mask = attention_mask, return_loss = True)

        img_outs = out['image_embeds']
        # img_outs = self.dropout(img_outs)
        img_outs = F.sigmoid(self.img_head(img_outs))

        txt_outs = out['text_embeds']
        # txt_outs = self.dropout(txt_outs)
        txt_outs = F.sigmoid(self.text_head(txt_outs))

        return out, img_outs, txt_outs

In [22]:
model= ContrastiveLearning().to(device)

In [23]:
torch.cuda.empty_cache()
weights = torch.tensor([0.5, 1., 1.25, 1.25, 1.25, 1.3, 1.25, .9]).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-6)
train_img_acc = MultilabelAccuracy(8, average = 'micro').to(device)
train_txt_acc = MultilabelAccuracy(8, average = 'micro').to(device)
val_img_acc = MultilabelAccuracy(8, average = 'micro').to(device)
val_txt_acc = MultilabelAccuracy(8, average = 'micro').to(device)

img_loss_fn = nn.BCELoss(weights)
txt_loss_fn = nn.BCELoss(weights)

EPOCHS = 100

for epoch_num in range(EPOCHS):

  total_acc_train = 0
  total_loss_train = 0

  for train_image, train_text, train_label in tqdm(train_dataloader):
        optimizer.zero_grad()
        train_label = train_label.to(device)
        train_image = train_image.to(device)
        mask = train_text['attention_mask'].to(device).squeeze(1)
        input_id = train_text['input_ids'].squeeze(1).to(device).squeeze(1)
        output, img_outs, txt_outs = model.forward(pixel_values = train_image, input_ids = input_id, attention_mask = mask)

        batch_loss = output['loss'] #+ img_loss_fn(img_outs, train_label) + txt_loss_fn(txt_outs, train_label)
        batch_loss.backward()
        optimizer.step()
        total_loss_train += batch_loss.item()
      
        train_img_acc(img_outs, train_label)
        train_txt_acc(txt_outs, train_label)

  total_loss_val = 0

  with torch.no_grad():
      #Validation
      for val_image, val_text, val_label in val_dataloader:

          val_label = val_label.to(device)
          val_image = val_image.to(device)
          mask = val_text['attention_mask'].to(device).squeeze(1)
          input_id = val_text['input_ids'].squeeze(1).to(device).squeeze(1)

          output, img_outs, txt_outs = model.forward(pixel_values = val_image, input_ids = input_id, attention_mask = mask)
          batch_loss = output['loss'] + img_loss_fn(img_outs, val_label) + txt_loss_fn(txt_outs, val_label)
          total_loss_val += batch_loss.item()
          
          val_img_acc(img_outs, val_label)
          val_txt_acc(txt_outs, val_label)
              
      
  avg_train_loss = total_loss_train/len(train_df)

  avg_val_loss = total_loss_val/len(val_df)


  print("Epoch [{}/{}], Train Loss: {:.4f}, acc img: {:.4f}, txt : {:.4f}".format(epoch_num+1, EPOCHS, avg_train_loss*BATCH_SIZE, train_img_acc.compute(), train_txt_acc.compute()))
  print("Epoch [{}/{}], Val Loss: {:.4f}, acc img: {:.4f}, txt : {:.4f}".format(epoch_num+1, EPOCHS, avg_val_loss*BATCH_SIZE, val_img_acc.compute(), val_txt_acc.compute()))

  train_img_acc.reset()
  train_txt_acc.reset()
  val_img_acc.reset()
  val_txt_acc.reset()

  torch.save(model.state_dict(), './' + 'checkpoint' + '.pt' )

torch.save(model.state_dict(), './' + 'finetuned' + '.pt' )


100%|██████████| 39/39 [00:22<00:00,  1.76it/s]


Epoch [1/100], Train Loss: 4.8960, acc img: 0.4004, txt : 0.5090
Epoch [1/100], Val Loss: 6.5442, acc img: 0.3884, txt : 0.5302


100%|██████████| 39/39 [00:19<00:00,  2.00it/s]


Epoch [2/100], Train Loss: 4.8751, acc img: 0.3878, txt : 0.5282
Epoch [2/100], Val Loss: 6.5286, acc img: 0.3942, txt : 0.5302


100%|██████████| 39/39 [00:19<00:00,  2.02it/s]


...

Epoch [6/100], Train Loss: 4.8445, acc img: 0.3718, txt : 0.5479
Epoch [6/100], Val Loss: 6.5166, acc img: 0.3647, txt : 0.5377


100%|██████████| 39/39 [00:19<00:00,  2.04it/s]


Epoch [7/100], Train Loss: 4.8455, acc img: 0.3622, txt : 0.5364
Epoch [7/100], Val Loss: 6.5024, acc img: 0.3582, txt : 0.5421


100%|██████████| 39/39 [00:19<00:00,  2.05it/s]


RuntimeError: CUDA error: device-side assert triggered
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
with open("finetuned.pt", 'rb') as f:
    model.load_state_dict(torch.load(f))

## Evaluation on test set

In [ ]:
#criterion = nn.BCELoss(torch.tensor([0.5, 1, 5, 5, 5, 6, 5, 1]).float().to(device))

test_loss = 0
test_acc  = 0

AVERAGING = 'weighted'
PREC = torchmetrics.classification.MultilabelPrecision(8, average = AVERAGING).to(device)#, validate_args = False)
ACC = torchmetrics.classification.MultilabelAccuracy(8, average = AVERAGING).to(device)#, validate_args = False)
REC = torchmetrics.classification.MultilabelRecall(8, average = AVERAGING).to(device)#, validate_args = False)
F1_SCORE = torchmetrics.classification.MultilabelF1Score(8, average = AVERAGING).to(device)#, validate_args = False)
F_BETA_SCORE = torchmetrics.classification.MultilabelFBetaScore(beta = 0.8, num_classes = 8, num_labels = 8, average = AVERAGING).to(device)#, validate_args = False)
KAPPA = torchmetrics.classification.MulticlassCohenKappa(8).to(device)#, validate_args = False)
AUC = torchmetrics.classification.MultilabelAUROC(8, average = AVERAGING).to(device)#, validate_args = False)

for train_image, train_text, train_label in tqdm(test_dataloader): 
    with torch.no_grad():
        train_label = train_label.to(device)
        train_image = train_image.to(device)
        mask = train_text['attention_mask'].to(device)
        input_id = train_text['input_ids'].squeeze(1).to(device)
        
        # logits_per_image, logits_per_text
        #out_img, out_txt = model.forward(train_image, input_id, mask, contrastive = True)
        output, predictions, txt_outs = model.forward(pixel_values = train_image, input_ids = input_id, attention_mask = mask)



        train_label = train_label.long()
        PREC(predictions, train_label)
        ACC(predictions, train_label)
        REC(predictions, train_label)
        F1_SCORE(predictions, train_label)
        F_BETA_SCORE(predictions, train_label)
        KAPPA(predictions, train_label)
        AUC(predictions, train_label)


add_prec = PREC.compute()
add_acc = ACC.compute()
add_rec = REC.compute()
add_f1 = F1_SCORE.compute()
add_fbeta = F_BETA_SCORE.compute()
add_kappa = KAPPA.compute()
add_auc = AUC.compute()

avg_test_loss = test_loss/len(test_df)*BATCH_SIZE
avg_test_acc  = test_acc /len(test_df)

print("Acc: {:3f}\nPrec: {:3f}\nRecall: {:.3f}\nF1-score: {:.3f}\nF-Beta-score: {:.3f}\nKappa: {:.3f}\nAUC: {:.3f}".format(add_acc, add_prec,add_rec, add_f1, add_fbeta, add_kappa, add_auc))
torch.cuda.empty_cache()

  0%|          | 0/27 [00:00<?, ?it/s]c:\ProgramData\Anaconda3\envs\torchnew\Lib\site-packages\torchmetrics\utilities\prints.py:36: UserWarning: No positive samples in targets, true positive value should be meaningless. Returning zero tensor in true positive score
  warnings.warn(*args, **kwargs)
100%|██████████| 27/27 [00:05<00:00,  4.50it/s]

Acc: 0.681057
Prec: 0.105590
Recall: 0.031
F1-score: 0.040
F-Beta-score: 0.044
Kappa: 0.000
AUC: 0.536
